# Open Library Search API Function
Resource: https://openlibrary.org/dev/docs/api/search

In [1]:
import requests
import pandas as pd

In [2]:
class OpenLibraryClient:
    """
    A client to interact with the Open Library Search API and 
    return data as structured DataFrames or Lists.
    """
    
    BASE_URL = "https://openlibrary.org/search.json"

    def __init__(self):
        self.session = requests.Session()

    def search(self, query=None, title=None, author=None, limit=10, advanced=True):
        """
        Executes search and returns a Pandas DataFrame with advanced fields.
        """
        # Define fields to fetch - (Added missing quotes from your draft)
        fields = [
            "key", "title", "author_name", "author_key", "first_publish_year",
            "edition_count", "subject", "ratings_average", "ratings_count",
            "readinglog_count", "want_to_read_count", "currently_reading_count",
            "already_read_count", "language", "number_of_pages_median",
            "first_sentence", "cover_i", "publisher", "publish_date", 
            "person", "place", "time"
        ]
        
        params = {
            "limit": limit,
            "fields": ",".join(fields)
        }
        
        if query: params["q"] = query
        if title: params["title"] = title
        if author: params["author"] = author

        try:
            response = self.session.get(self.BASE_URL, params=params)
            response.raise_for_status()
            docs = response.json().get('docs', [])
            
            return self._process_to_dataframe(docs)
            
        except requests.exceptions.RequestException as e:
            print(f"API Error: {e}")
            return pd.DataFrame()

    def _process_to_dataframe(self, docs):
        """Internal helper to clean data into a table format."""
        rows = []
        for doc in docs:
            # Helper to join list fields into strings for easier NLP cleaning later
            def safe_join(field_name):
                val = doc.get(field_name, [])
                return ", ".join(val) if isinstance(val, list) else str(val)

            rows.append({
                "title": doc.get("title"),
                "author": safe_join("author_name"),
                "author_key": safe_join("author_key"),
                "year": doc.get("first_publish_year"),
                "editions": doc.get("edition_count", 0),
                "rating_avg": round(doc.get("ratings_average", 0), 2) if doc.get("ratings_average") else None,
                "rating_count": doc.get("ratings_count", 0),
                "want_to_read": doc.get("want_to_read_count", 0),
                "currently_reading": doc.get("currently_reading_count", 0),
                "already_read": doc.get("already_read_count", 0),
                "readinglog_count": doc.get("readinglog_count", 0),
                "pages": doc.get("number_of_pages_median"),
                "language": safe_join("language"),
                "publisher": safe_join("publisher"),
                "subjects": safe_join("subject"),
                "people": safe_join("person"),
                "places": safe_join("place"),
                "times": safe_join("time"),
                "first_sentence": doc.get("first_sentence"),
                "cover_url": f"https://covers.openlibrary.org/b/id/{doc.get('cover_i')}-L.jpg" if doc.get("cover_i") else None,
                "work_key": doc.get("key")
            })
            
        df = pd.DataFrame(rows)
        
        # --- FEATURE ENGINEERING FOR YOUR APP ---
        # Create a 'popularity_score' for your prediction model
        if not df.empty:
            df['popularity_score'] = df['rating_count'] + df['want_to_read'] + df['already_read']
            
        return df

In [3]:
# Create a default instance for easy importing
client = OpenLibraryClient()

In [4]:
# This looks at the file above and grabs the 'client' object
# from open_library_client import client 

# Now you can use it!
df = client.search(query="harry potter")
df

,title,author,author_key,year,editions,rating_avg,rating_count,want_to_read,currently_reading,already_read,...,language,publisher,subjects,people,places,times,first_sentence,cover_url,work_key,popularity_score
0,Harry Potter and the Order of the Phoenix,J.K. Rowling,OL23919A,2003,226,4.22,302,3364,285,632,...,"eng, ben, heb, bul, tur, fre, ger, hun, dan, p...","Aṅkura Prakāśanī, Seizansha, Gallimard, Listen...","Children's Books/Ages 9-12 Fiction, Witches an...","Harry Potter, Albus Dumbledore, Hagrid, Ron We...","England, Hogwarts Schools of Witchcraft and Wi...",,[Il giorno più caldo dell’estate – almeno fino...,https://covers.openlibrary.org/b/id/10523466-L...,/works/OL82548W,4298
1,Harry Potter and the Philosopher's Stone,J.K. Rowling,OL23919A,1997,357,4.24,941,18313,1624,1515,...,"urd, lat, hun, por, jpn, ltz, heb, ita, tha, b...","Bloomsbury Publishing (IN), Salamandra, Emece ...","series:Harry_Potter, Ghosts, Monsters, Vampire...","Harry Potter, Ron Weasley, Hermione Granger, N...","England, Hogwarts School of Witchcraft and Wiz...",Late 1990s,"[O Sr. e a Sra. Dursley, da rua dos Alfeneiros...",https://covers.openlibrary.org/b/id/10521270-L...,/works/OL82563W,20769
2,Harry Potter and the Deathly Hallows,J.K. Rowling,OL23919A,2007,126,4.27,400,4565,336,717,...,"hun, por, jpn, heb, ita, ben, bul, spa, rus, h...","Salamandra, Editorial Presença, BLOOMSBURY, Ed...","the Elder Wand, children's books, dementors, g...","Harry Potter, Gregorovitch, Greyback, Gellert ...","England, Scottland, Ireland, Wales, Hogwarts S...",1987-,"[Les deux hommes surgirent de nulle part, à qu...",https://covers.openlibrary.org/b/id/10110415-L...,/works/OL82586W,5682
3,Harry Potter and the Prisoner of Azkaban,J.K. Rowling,OL23919A,1999,263,4.25,608,3929,287,1068,...,"eng, ben, alb, heb, bul, tur, fre, ger, per, j...","Aṅkura Prakāśanī, Seizansha, Gallimard, Editur...","Fantasy fiction, orphans, foster homes, fantas...","Harry Potter, Hermione Granger, Ron Weasley, S...","England, Azkaban, Hogwarts School of Witchcraf...",,[Harry Potter era un ragazzo insolito sotto mo...,https://covers.openlibrary.org/b/id/10580435-L...,/works/OL82536W,5605
4,Harry Potter and the Chamber of Secrets,J.K. Rowling,OL23919A,1998,276,4.20,441,4890,428,867,...,"spa, tha, chi, rus, vie, dan, ara, ger, pol, h...",Arthur A. Levine Books: An Imprint of Scholast...,"series:Harry_Potter, Fantasy fiction, school s...","Harry Potter, Hermione Granger, Ron Weasley, A...","England, London, Hogwarts School of Witchcraft...",Late 1990s,[Ce n’était pas la première fois qu’une disput...,https://covers.openlibrary.org/b/id/8392798-L.jpg,/works/OL82537W,6198
5,Harry Potter and the Half-Blood Prince,"J.K. Rowling, Mary GrandPré","OL23919A, OL2703953A",2005,151,4.36,198,2751,196,484,...,"hun, por, jpn, heb, ita, ben, bul, spa, rus, h...","Salamandra, Lulu Press, Inc., Arthur A. Levine...","orphans, foster homes, romans, magie, adolesce...","Harry Potter, Dumbledore, Lord Voldemort, Drac...","England, Hogwarts School of Witchcraft and Wiz...",,[Il était près de minuit et le Premier Ministr...,https://covers.openlibrary.org/b/id/10716273-L...,/works/OL82565W,3433
6,Harry Potter and the Goblet of Fire,J.K. Rowling,OL23919A,2000,231,4.27,327,3655,316,649,...,"eng, ben, alb, heb, bul, tur, fre, ger, per, d...","Aṅkura Prakāśanī, Seizansha, Gallimard, Listen...","orphans, foster homes, Waisenkind, Intrige, Wi...","Harry Potter, Dumbledore, Hagrid, Ron Weasley,...","England, Angleterre, Hogwarts School of Witchc...",2000-2009,"[oopsy i farted, The villagers of Little Hangl...",https://covers.openlibrary.org/b/id/12059372-L...,/works/OL82560W,4631
7,Harry Potter,Warner Bros. Entertainment Inc,OL7983102A,2007,1,3.88,8,217,21,10,...,eng,"Warner Bros, Egmont UK Ltd.","Motion pictures, Hogwarts School of Witchcraft...",Harry Potter (Fictitious character),,,None,https://covers.openlibrary.org/b/id/10195373-L...,/works/OL20874116W,235
8,Harry Potter (series) 1-7,J.K. Rowling,OL23919A,1999,16,4.62,47,

In [5]:
df.to_csv('Test_API_Sample.csv')